In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import random
from scipy import stats
import datetime
import calendar
import math
import six

In [ ]:
# This class generates the daily, close-to-close profit and loss of simple option strategies like (short, long)...
#...(ATM, OTM, ITM) put or call, credit and debit spreads, strangles and straddles. The strikes are chosen based on the...
#...close each day. 

class OptionStrategyPnlGenerator():

    def __init__(self, location_dict, entry_time='close', strategy = 'short-naked-put-atm-0', straddle_df = None, symbol = None, weekly = 1, monthly = 1, holding_period=1, setToCurrentExpiry=0, setToTradedExpiry=0, exitat='close', on_future=False, cost=0):

        # We start with the strategy name that is passed through when an object of this class is created
        self.strategy  = strategy
        self.straddle_df = straddle_df
        self.location_dict = location_dict
        self.entry_time = entry_time
        self.on_future=on_future
        self.symbol = symbol
        self.weekly = weekly
        self.monthly = monthly
        self.holding_period = holding_period
        self.setToCurrentExpiry = setToCurrentExpiry
        self.setToTradedExpiry = setToTradedExpiry
        self.exitat = exitat
        self.cost = cost
        
        self.daily_options_data = pd.read_pickle(self.location_dict['option'][self.entry_time]) 
        
    def load_data(self):
        
        print('reading the data')
        print()
        # Then we get the daily spot price and the daily options data for all strikes and expiries
        
        self.eod_data = pd.read_pickle(self.location_dict[self.expiry][self.entry_time])                   
        
    def get_weekly_expiry_dates(self, end_date=None):
        
        print('setting the date range')
        print()
        if self.symbol == 'banknifty':
            self.eod_data = self.eod_data[self.eod_data['date'] >= datetime.date(2016, 6, 1)]
            self.straddle_df = self.straddle_df[self.straddle_df['date'] >= datetime.date(2016, 6, 1)]
            self.eod_data.reset_index(inplace = True, drop = True)
            self.straddle_df.reset_index(inplace = True, drop = True)
            self.straddle_df = self.straddle_df[['date', f'{self.expiry}_{self.entry_time}_straddle_price']]
            self.eod_data = self.eod_data.merge(self.straddle_df, on='date', how='left')
            if end_date != None:
                self.eod_data = self.eod_data[self.eod_data['date'] <= end_date]
            
        else:
            self.eod_data = self.eod_data[self.eod_data['date'] >= datetime.date(2019, 2, 1)]
            self.straddle_df = self.straddle_df[self.straddle_df['date'] >= datetime.date(2019, 2, 1)]
            self.eod_data.reset_index(inplace = True, drop = True)
            self.straddle_df.reset_index(inplace = True, drop = True)
            self.eod_data['weekly_straddle'] = self.straddle_df['weekly_straddle_price']
            self.eod_data['monthly_straddle'] = self.straddle_df['monthly_straddle_price']
            if end_date != None:
                self.eod_data = self.eod_data[self.eod_data['date'] <= end_date]
    
    def get_monthly_straddle_column(self):
        
        date = self.straddle_df['date'].iloc[0]
        straddle_monthly = self.straddle_df[['date', 'monthly_straddle_price']]
        straddle_monthly.columns = 'date monthly_straddle'.split()
        self.eod_data = self.eod_data.merge(straddle_monthly, on = 'date', how = 'left')
        self.eod_data = self.eod_data[self.eod_data['date'] >= date]
    
    def set_backtest_period(self, date=None):
        
        self.eod_data = self.eod_data[self.eod_data['date'] >= date]
    
    def days_remaining_in_month(self):
        
        self.eod_data['days_remain_month'] = self.eod_data['date'].apply(lambda x: calendar.monthrange(x.year, x.month)[1] - x.day)
    
    def get_business_days_remaining_month(self):
    
        self.eod_data['day_of_month'] = 0
        for _ in range(1, len(self.eod_data)):

            if self.eod_data['date'].iloc[_].month != self.eod_data['date'].iloc[_ - 1].month:
                self.eod_data['day_of_month'].iloc[_] = 1

            else:
                self.eod_data['day_of_month'].iloc[_] = 1 + self.eod_data['day_of_month'].iloc[_ - 1]

        self.eod_data['business_days_remaining_in_month'] = np.where(self.eod_data['day_of_month'] > self.eod_data['day_of_month'].shift(-1), self.eod_data['day_of_month'], np.nan)

        self.eod_data['business_days_remaining_in_month'].bfill(inplace = True)

        self.eod_data = self.eod_data[~self.eod_data['business_days_remaining_in_month'].isnull()]

        self.eod_data['business_days_remaining_in_month'] = self.eod_data['business_days_remaining_in_month'] - self.eod_data['day_of_month']
        
    def get_seasonality_eod_data(self, days_to_month_end=5):
        
        self.days_remaining_in_month()
        self.eod_data['close_last_day_of_month'] = np.where(self.eod_data['business_days_remaining_in_month'].shift(-1) > self.eod_data['business_days_remaining_in_month'], self.eod_data['close'], np.nan)
        self.eod_data['close_last_day_of_month'].bfill(inplace = True)
        self.eod_data['day_of_month_pnl'] = self.eod_data['close_last_day_of_month'] - self.eod_data['close']
        
        self.eod_data = self.eod_data[self.eod_data['business_days_remaining_in_month'] <= days_to_month_end]

        self.eod_data['last_date'] = np.where(self.eod_data['business_days_remaining_in_month'].shift(-1) > self.eod_data['business_days_remaining_in_month'], self.eod_data['date'], np.nan)
        self.eod_data['last_date'].bfill(inplace = True)

        self.eod_data['t+1'] = np.where(self.eod_data['current_week_expiry'] < self.eod_data['last_date'], self.eod_data['current_week_expiry'], self.eod_data['last_date'])

        self.eod_data = self.eod_data.drop_duplicates(subset='t+1', keep="first")
    
    def set_holding_period(self):
        
        print('Setting the holding period')
        print()
        if self.expiry == 'weekly':
            self.eod_data['expiry'] = np.where(self.eod_data['current_week_expiry'] != self.eod_data['current_week_expiry'].shift(), self.eod_data['current_week_expiry'].shift(), self.eod_data['current_week_expiry'])
            shiftby = -1*self.holding_period
            self.eod_data[f't+{self.holding_period}'] = self.eod_data['date'].shift(shiftby)
            self.eod_data[f't+{self.holding_period}_expiry'] = self.eod_data['expiry'].shift(shiftby)
            if self.setToCurrentExpiry == 1:
                self.eod_data[f't+{self.holding_period}'] = np.where(self.eod_data['current_week_expiry'] == self.eod_data[f't+{self.holding_period}_expiry'], self.eod_data[f't+{self.holding_period}'], self.eod_data['current_week_expiry'])
                self.eod_data['traded_expiry'] = self.eod_data['current_week_expiry']
            if self.setToTradedExpiry == 1:
                self.eod_data[f't+{self.holding_period}'] = self.eod_data[f't+{self.holding_period}_expiry']
                self.eod_data['traded_expiry'] = self.eod_data[f't+{self.holding_period}_expiry']
            if self.setToCurrentExpiry == 0 and self.setToTradedExpiry == 0:
                self.eod_data['traded_expiry'] = self.eod_data[f't+{self.holding_period}_expiry']
                
        else:
            self.eod_data['expiry'] = np.where(self.eod_data['current_month_expiry'] != self.eod_data['current_month_expiry'].shift(), self.eod_data['current_month_expiry'].shift(), self.eod_data['current_month_expiry'])
            shiftby = -1*self.holding_period
            self.eod_data[f't+{self.holding_period}'] = self.eod_data['date'].shift(shiftby)
            self.eod_data[f't+{self.holding_period}_expiry'] = self.eod_data['expiry'].shift(shiftby)
            if self.setToCurrentExpiry == 1:
                self.eod_data[f't+{self.holding_period}'] = np.where(self.eod_data['current_month_expiry'] == self.eod_data[f't+{self.holding_period}_expiry'], self.eod_data[f't+{self.holding_period}'], self.eod_data['current_month_expiry'])
                self.eod_data['traded_expiry'] = self.eod_data['current_month_expiry']
            if self.setToTradedExpiry == 1:
                self.holding_period = str(self.holding_period) + '_expiry'
                self.eod_data[f't+{self.holding_period}'] = self.eod_data[f't+{self.holding_period}_expiry']
                self.eod_data['traded_expiry'] = self.eod_data[f't+{self.holding_period}_expiry']
            if self.setToCurrentExpiry == 0 and self.setToTradedExpiry == 0:
                self.eod_data['traded_expiry'] = self.eod_data[f't+{self.holding_period}_expiry']
        
    def generate_option_strategy_pnl(self):
        
        print(f'Getting PnL for {self.strategy} in the daily timeframe.')
        # In the daily options data, we create  a new column called lookup that combines the date, option type, strike...
        #... and expiry in the string format with a '-' seperating each column value
        columns = ['date', 'option_type', 'strike', 'expiry']
        self.daily_options_data['lookup'] = self.daily_options_data[columns].apply(lambda row: '-'.join(row.values.astype(str)), axis = 1)
    
        # After that has been done, we need to work with the eod data to get the strike price based on each day's close
        # So we check if the strategy is a single leg or naked strategy
        if 'naked' in self.strategy.split('-'):

            # Then we check whether the strategy is long or short
            self.position = self.strategy.split('-')[0]

            # Then we check the option type (call or put)
            self.option_type = self.strategy.split('-')[2]

            # This the string 'atm' or 'itm' or 'otm'
            self.strike = self.strategy.split('-')[3]

            # If it is atm, this will be 0. With 'itm' or 'otm', this will select strike the specified points away from atm
            self.points_from_atm = self.strategy.split('-')[4]
             
            self.expiry = self.strategy.split('-')[5]
            
            self.load_data()
            self.get_weekly_expiry_dates()
            
            self.base = int(self.strategy.split('-')[6])
            
            # New column in eod_data called option type which will have call or put based on the strategy
            self.eod_data['option_type'] = self.option_type
            
            self.set_holding_period()
            
            if (self.weekly == 1 and self.expiry == 'weekly') or (self.monthly == 1 and self.expiry == 'monthly'):

                print('getting the strike')
                # Calculates the strike at the close everyday
                if self.points_from_atm == 'fullstraddle':
                    self.get_fullstraddle_strike()
                elif self.points_from_atm == 'halfstraddle':
                    self.get_halfstraddle_strike()
                else:
                    self.get_naked_strike()

                # Create a list of column names that we have to combine to create a lookup column. The entry column is for...
                #... today's date and the exit column will have the next trading day's date.
                columns_entry = ['date', 'option_type', 'strike', 'traded_expiry']
                columns_exit = [f't+{self.holding_period}', 'option_type', 'strike', 'traded_expiry']                

                print('joining columns to create lookup...')
                # So, now we have two columns, the entry_lookup and the exit_lookup
                self.join_columns_to_create_lookup(columns_entry, columns_exit, 'entry_lookup', 'exit_lookup')

                # Select the chosen columns from eod_data to create a seperate dataframe called options_pnl
                self.options_pnl = self.eod_data[['date', 'close', 'strike', f't+{self.holding_period}', 'traded_expiry',\
                                                                      'entry_lookup', 'exit_lookup']]

                # From the daily options dataframe, create a new dataframe call options_data_lookup for merging which has...
                #...only the close and lookup columns
                self.options_data_lookup = self.daily_options_data[['close', 'lookup']]
                self.options_pnl_test = self.options_pnl.copy()
                self.options_data_test = self.options_data_lookup.copy()
                print('merging the columns...')
                # Rename the columns in options_data_lookup into entry and enty_lookup and then merge it into options_pnl df 
                self.lookup('entry entry_lookup')

                # Rename the columns in options_data_lookup into exit and exit_lookup and then merge it into options_pnl df
                self.lookup('exit exit_lookup')

                print('Replacing NaNs...')
                # So, now we have the entry price and exit price for the selected strike, option type and the expiry
                # Then we have to check whether the entry and exit columns have any NaN values. This will happen if the selected...
                #... strike did not trade on that day. Apart from the last row, if there are any NaN values, we have to replace...
                #... them with close of traded strikes that are the closest to the close of the day
                self.replace_nan('entry entry_lookup', 'exit exit_lookup', 'strike')

                print('Calculating pnl...')
                # If the position is short, do entry - exit and if it is long, do exit - entry
                if self.position == 'short':

                    self.options_pnl['pnl'] = (self.options_pnl['entry']*(1-self.cost)) - (self.options_pnl['exit']*(1+self.cost))

                else:

                    self.options_pnl['pnl'] = (self.options_pnl['exit']*(1-self.cost)) - (self.options_pnl['entry']*(1+self.cost))

                # Create a new dataframe called strategy_pnl_df to get only the date, pnl and the strategy name. This dataframe...
                #... will be used to create the strategy library.
                print('getting new dataframe...')
                print()
                self.strategy_pnl_df = self.options_pnl[['date', 'entry', 'pnl']]
                self.option_pnl_df = self.options_pnl[['date', f't+{self.holding_period}', 'pnl']]
                self.option_pnl_df = self.option_pnl_df.rename(columns={'date':'entry_date', f't+{self.holding_period}':'exit_date'})
                self.option_pnl_df['strategy'] = self.strategy
                self.strategy_pnl_df['strategy'] = self.strategy
                self.strategy_pnl_df.dropna(inplace = True)
                self.option_pnl_df.dropna(inplace=True)
                
                print('Done')
                print()
            
        # Here you repeat the entire procedure if the strategy is spread. The difference from the above is that here there...
        #... are two legs instead of one above. One leg will be short and the other will be long
        elif 'spread' in self.strategy.split('-'):

            self.position = self.strategy.split('-')[1]
            self.option_type = self.strategy.split('-')[0]
            self.strike = 'atm'
            self.points_from_atm = self.strategy.split('-')[3]
            self.expiry = self.strategy.split('-')[4]
            self.load_data()
            self.get_weekly_expiry_dates()
            self.base = int(self.strategy.split('-')[5])
            self.eod_data['option_type'] = self.option_type
            
            self.set_holding_period()
            
            if (self.weekly == 1 and self.expiry == 'weekly') or (self.monthly == 1 and self.expiry == 'monthly'):
                print('getting the spread strikes...')
                # We have to get two strikes. 
                self.get_spread_strikes()

                columns_entry_nearstrike = ['date', 'option_type', 'nearstrike', 'traded_expiry']
                columns_exit_nearstrike = [f't+{self.holding_period}', 'option_type', 'nearstrike', 'traded_expiry']

                print('joining columns to create lookup...')
                self.join_columns_to_create_lookup(columns_entry_nearstrike, columns_exit_nearstrike, 'entry_lookup_nearstrike',\
                                                   'exit_lookup_nearstrike')

                columns_entry_farstrike = ['date', 'option_type', 'farstrike', 'traded_expiry']
                columns_exit_farstrike = [f't+{self.holding_period}', 'option_type', 'farstrike', 'traded_expiry']

                self.join_columns_to_create_lookup(columns_entry_farstrike, columns_exit_farstrike, 'entry_lookup_farstrike',\
                                                   'exit_lookup_farstrike')

                self.options_pnl = self.eod_data[['date', 'close', 'nearstrike', 'farstrike', f't+{self.holding_period}', 'traded_expiry',\
                                              'entry_lookup_nearstrike', 'exit_lookup_nearstrike', 'entry_lookup_farstrike',\
                                              'exit_lookup_farstrike']]

                self.options_data_lookup = self.daily_options_data[['close', 'lookup']]

                print('merging the columns...')
                self.lookup('entry_nearstrike entry_lookup_nearstrike')

                self.lookup('exit_nearstrike exit_lookup_nearstrike')

                self.lookup('entry_farstrike entry_lookup_farstrike')

                self.lookup('exit_farstrike exit_lookup_farstrike')

                print('Replacing NaNs...')

                self.replace_nan('entry_nearstrike entry_lookup_nearstrike', 'exit_nearstrike exit_lookup_nearstrike', 'nearstrike')

                self.replace_nan('entry_farstrike entry_lookup_farstrike', 'exit_farstrike exit_lookup_farstrike', 'farstrike')

                print('Calculating pnl...')
                # If it is a credit spread, calculate the pnl from close to close and store in a column 'pnl'. Since it is a short 
                # position, do the entry price - exit price for the near strike and the opposite for the far strike
                if self.position == 'credit':

                    self.options_pnl['pnl_nearstrike'] = (self.options_pnl['entry_nearstrike']*(1-self.cost)) - (self.options_pnl['exit_nearstrike']*(1+self.cost))
                    self.options_pnl['pnl_farstrike'] = (self.options_pnl['exit_farstrike']*(1-self.cost)) - (self.options_pnl['entry_farstrike']*(1+self.cost))
                    self.options_pnl['pnl'] = self.options_pnl['pnl_nearstrike'] + self.options_pnl['pnl_farstrike']

                # Else, if it is a debit spread, do the same. Since it is a long position, do the exit price - entry price for the
                # near strike and the opposite for the far strike
                else:

                    self.options_pnl['pnl_nearstrike'] = (self.options_pnl['exit_nearstrike']*(1-self.cost)) - (self.options_pnl['entry_nearstrike']*(1+self.cost))
                    self.options_pnl['pnl_farstrike'] = (self.options_pnl['entry_farstrike']*(1-self.cost)) - (self.options_pnl['exit_farstrike']*(1+self.cost))
                    self.options_pnl['pnl'] = self.options_pnl['pnl_nearstrike'] + self.options_pnl['pnl_farstrike']

                print('getting new dataframe...')
                print()
                # Then you select only the date and pnl column and store in a strategy_pnl dataframe
                self.strategy_pnl_df = self.options_pnl[['date', 'pnl']]

                # Also, include the strategy name in a strategy column
                self.strategy_pnl_df['strategy'] = self.strategy
                self.strategy_pnl_df.dropna(inplace = True)
                self.option_pnl_df = self.options_pnl[['date', f't+{self.holding_period}', 'pnl']]
                self.option_pnl_df = self.option_pnl_df.rename(columns={'date':'entry_date', f't+{self.holding_period}':'exit_date'})
                self.option_pnl_df['strategy'] = self.strategy
                self.option_pnl_df.dropna(inplace=True)
                
                print('Done')
                print()
            
        elif 'strangle' in self.strategy.split('-'):
            
            self.position = self.strategy.split('-')[0]
            self.strike = 'atm'
            self.points_from_atm = self.strategy.split('-')[2]
            self.expiry = self.strategy.split('-')[3]
            self.load_data()
            self.get_weekly_expiry_dates()
            self.base = int(self.strategy.split('-')[4])
            self.set_holding_period()
            
            if (self.weekly == 1 and self.expiry == 'weekly') or (self.monthly == 1 and self.expiry == 'monthly'):
            
                if self.points_from_atm == 'fullstraddle':
                    self.get_fullstraddle_strangle_strikes()
                elif self.points_from_atm == 'halfstraddle':
                    self.get_halfstraddle_strangle_strikes()
                else:
                    self.get_absolute_strangle_strikes()

                self.eod_data['leg_one_option_type'] = 'call'
                self.eod_data['leg_two_option_type'] = 'put'

                print('joining columns to create lookup')
                print()
                columns_leg_one_entry = ['date', 'leg_one_option_type', 'leg_one_strike', 'traded_expiry']
                columns_leg_one_exit = [f't+{self.holding_period}', 'leg_one_option_type', 'leg_one_strike', 'traded_expiry']
                columns_leg_two_entry = ['date', 'leg_two_option_type', 'leg_two_strike', 'traded_expiry']
                columns_leg_two_exit = [f't+{self.holding_period}', 'leg_two_option_type', 'leg_two_strike', 'traded_expiry']


                self.join_columns_to_create_lookup(columns_leg_one_entry, columns_leg_one_exit, \
                                                   'leg_one_entry_lookup', 'leg_one_exit_lookup')            

                self.join_columns_to_create_lookup(columns_leg_two_entry, columns_leg_two_exit, \
                                                   'leg_two_entry_lookup', 'leg_two_exit_lookup')

                self.options_pnl = self.eod_data[['date', 'close', 'leg_one_strike', 'leg_two_strike', f't+{self.holding_period}', 'traded_expiry',\
                                              'leg_one_entry_lookup', 'leg_one_exit_lookup', 'leg_two_entry_lookup', 'leg_two_exit_lookup']]

                self.options_data_lookup = self.daily_options_data[['close', 'lookup']]

                print('merging the columns')
                print()
                self.lookup('entry_leg_one leg_one_entry_lookup')

                self.lookup('exit_leg_one leg_one_exit_lookup')

                self.lookup('entry_leg_two leg_two_entry_lookup')

                self.lookup('exit_leg_two leg_two_exit_lookup')

                print('Replacing NaNs')
                print()
                self.option_type = 'call'

                self.replace_nan('entry_leg_one leg_one_entry_lookup', 'exit_leg_one leg_one_exit_lookup', 'leg_one_strike')

                self.option_type = 'put'

                self.replace_nan('entry_leg_two leg_two_entry_lookup', 'exit_leg_two leg_two_exit_lookup', 'leg_two_strike')

                print('Calculating pnl')
                print()
                if self.position == 'short':

                    self.options_pnl['pnl_leg_one'] = (self.options_pnl['entry_leg_one']*(1-self.cost)) - (self.options_pnl['exit_leg_one']*(1+self.cost))
                    self.options_pnl['pnl_leg_two'] = (self.options_pnl['entry_leg_two']*(1-self.cost)) - (self.options_pnl['exit_leg_two']*(1+self.cost))
                    self.options_pnl['pnl'] = self.options_pnl['pnl_leg_one'] + self.options_pnl['pnl_leg_two']

                if self.position == 'long':

                    self.options_pnl['pnl_leg_one'] = (self.options_pnl['exit_leg_one']*(1-self.cost)) - (self.options_pnl['entry_leg_one']*(1+self.cost))
                    self.options_pnl['pnl_leg_two'] = (self.options_pnl['exit_leg_two']*(1-self.cost)) - (self.options_pnl['entry_leg_two']*(1+self.cost))
                    self.options_pnl['pnl'] = self.options_pnl['pnl_leg_one'] + self.options_pnl['pnl_leg_two']
                    self.options_pnl['entry'] = self.options_pnl['entry_leg_one'] + self.options_pnl['entry_leg_two']

                print('getting new dataframe')
                print()
                # Then you select only the date and pnl column and store in a strategy_pnl dataframe
                if self.position == 'long':
                    self.strategy_pnl_df = self.options_pnl[['date', 'entry', 'pnl']]
                else:
                    self.strategy_pnl_df = self.options_pnl[['date', 'pnl']]

                # Also, include the strategy name in a strategy column
                self.strategy_pnl_df['strategy'] = self.strategy
                self.strategy_pnl_df.dropna(inplace = True)
                self.option_pnl_df = self.options_pnl[['date', f't+{self.holding_period}', 'pnl']]
                self.option_pnl_df = self.option_pnl_df.rename(columns={'date':'entry_date', f't+{self.holding_period}':'exit_date'})
                self.option_pnl_df['strategy'] = self.strategy
                self.option_pnl_df.dropna(inplace=True)
                
                print('Done.')
                print()

        elif 'straddle' in self.strategy.split('-'):

            self.position = self.strategy.split('-')[0]
            if self.strategy.split('-')[2] == 'atm':
                self.strike = 'atm'
            elif self.strategy.split('-')[2] == 'zerodelta':
                self.strike = 'zerodelta'
                
            self.expiry = self.strategy.split('-')[3]
            self.load_data()
            self.get_weekly_expiry_dates()
            self.base = int(self.strategy.split('-')[4])
            
            self.eod_data['leg_one_option_type'] = 'call'
            self.eod_data['leg_two_option_type'] = 'put'
            
            self.set_holding_period()
            
            if (self.weekly == 1 and self.expiry == 'weekly') or (self.monthly == 1 and self.expiry == 'monthly'):
                if self.strike == 'atm':
                    print('getting the atm straddle strikes...')
                    self.eod_data['leg_one_strike'] = (round(self.eod_data['close']/self.base) * self.base)
                    self.eod_data['leg_one_strike'] = self.eod_data['leg_one_strike'].astype(float)
                    self.eod_data['leg_two_strike'] = (round(self.eod_data['close']/self.base) * self.base)
                    self.eod_data['leg_two_strike'] = self.eod_data['leg_two_strike'].astype(float)
                elif self.strike == 'zerodelta':
                    print('getting the atm straddle strikes...')
                    self.eod_data['leg_one_strike'] = (round(self.eod_data['close']/self.base) * self.base)
                    self.eod_data['leg_one_strike'] = self.eod_data['leg_one_strike'].astype(float)
                    self.eod_data['leg_two_strike'] = (round(self.eod_data['close']/self.base) * self.base)
                    self.eod_data['leg_two_strike'] = self.eod_data['leg_two_strike'].astype(float)

                print('joining columns to create lookup for leg one strike...')
                columns_leg_one_entry = ['date', 'leg_one_option_type', 'leg_one_strike', 'traded_expiry']
                columns_leg_one_exit = [f't+{self.holding_period}', 'leg_one_option_type', 'leg_one_strike', 'traded_expiry']

                self.join_columns_to_create_lookup(columns_leg_one_entry, columns_leg_one_exit, \
                                                   'leg_one_entry_lookup', 'leg_one_exit_lookup')

                print('joining columns to create lookup for leg two strike...')
                columns_leg_two_entry = ['date', 'leg_two_option_type', 'leg_two_strike', 'traded_expiry']
                columns_leg_two_exit = [f't+{self.holding_period}', 'leg_two_option_type', 'leg_two_strike', 'traded_expiry']

                self.join_columns_to_create_lookup(columns_leg_two_entry, columns_leg_two_exit, \
                                                   'leg_two_entry_lookup', 'leg_two_exit_lookup')


                self.options_pnl = self.eod_data[['date', 'close', 'leg_one_strike', 'leg_two_strike', f't+{self.holding_period}', 'traded_expiry',\
                                              'leg_one_entry_lookup', 'leg_one_exit_lookup', 'leg_two_entry_lookup', 'leg_two_exit_lookup']]


                self.options_data_lookup = self.daily_options_data[['close', 'lookup']]

                print('merging the columns...')
                self.lookup('entry_leg_one leg_one_entry_lookup')

                self.lookup('exit_leg_one leg_one_exit_lookup')

                self.lookup('entry_leg_two leg_two_entry_lookup')

                self.lookup('exit_leg_two leg_two_exit_lookup')

                print('Replacing NaNs...')
                self.option_type = 'call'

                self.replace_nan('entry_leg_one leg_one_entry_lookup', 'exit_leg_one leg_one_exit_lookup', 'leg_one_strike')

                self.option_type = 'put'

                self.replace_nan('entry_leg_two leg_two_entry_lookup', 'exit_leg_two leg_two_exit_lookup', 'leg_two_strike')

                print('Calculating pnl...')
                if self.position == 'short':

                    self.options_pnl['pnl_leg_one'] = (self.options_pnl['entry_leg_one']*(1-self.cost)) - (self.options_pnl['exit_leg_one']*(1+self.cost))
                    self.options_pnl['pnl_leg_two'] = (self.options_pnl['entry_leg_two']*(1-self.cost)) - (self.options_pnl['exit_leg_two']*(1+self.cost))
                    self.options_pnl['pnl'] = self.options_pnl['pnl_leg_one'] + self.options_pnl['pnl_leg_two']

                if self.position == 'long':

                    self.options_pnl['pnl_leg_one'] = self.options_pnl['exit_leg_one'] - self.options_pnl['entry_leg_one']
                    self.options_pnl['pnl_leg_two'] = self.options_pnl['exit_leg_two'] - self.options_pnl['entry_leg_two']
                    self.options_pnl['pnl'] = self.options_pnl['pnl_leg_one'] + self.options_pnl['pnl_leg_two']
                    self.options_pnl['entry'] = self.options_pnl['entry_leg_one'] + self.options_pnl['entry_leg_two']

                print('getting new dataframe...')
                print()
                # Then you select only the date and pnl column and store in a strategy_pnl dataframe
                if self.position == 'long':
                    self.strategy_pnl_df = self.options_pnl[['date', 'entry', 'pnl']]
                else:
                    self.strategy_pnl_df = self.options_pnl[['date', 'pnl']]

                # Also, include the strategy name in a strategy column
                self.strategy_pnl_df['strategy'] = self.strategy
                self.strategy_pnl_df.dropna(inplace = True)
                self.option_pnl_df = self.options_pnl[['date', f't+{self.holding_period}', 'pnl']]
                self.option_pnl_df = self.option_pnl_df.rename(columns={'date':'entry_date', f't+{self.holding_period}':'exit_date'})
                self.option_pnl_df['strategy'] = self.strategy
                self.option_pnl_df.dropna(inplace=True)
                
                print('Done')
                print()
            
    def get_fullstraddle_strike(self):
        
        if self.option_type == 'put':

            if self.strike == 'otm':
                if self.expiry == 'weekly':
                    print('getting weekly atm strikes with fullstraddle premium away')
                    print()
                    self.eod_data['strike'] = self.eod_data['close'] - self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price']
                    self.eod_data['strike'] = (round(self.eod_data['strike']/self.base) * self.base)
                    self.eod_data['strike'] = self.eod_data['strike'].astype(float)
                elif self.expiry == 'monthly':
                    print('getting monthly atm strikes with fullstraddle premium away')
                    print()
                    self.eod_data['strike'] = self.eod_data['close'] - self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price']
                    self.eod_data['strike'] = (round(self.eod_data['strike']/self.base) * self.base)
                    self.eod_data['strike'] = self.eod_data['strike'].astype(float)
            
            elif self.strike == 'itm':
                if self.expiry == 'weekly':
                    print('getting weekly otm strikes with fullstraddle premium away')
                    print()
                    self.eod_data['strike'] = self.eod_data['close'] + self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price']
                    self.eod_data['strike'] = (round(self.eod_data['strike']/self.base) * self.base)
                    self.eod_data['strike'] = self.eod_data['strike'].astype(float)
                elif self.expiry == 'monthly':
                    print('getting monthly otm strikes with fullstraddle premium away')
                    print()
                    self.eod_data['strike'] = self.eod_data['close'] + self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price']
                    self.eod_data['strike'] = (round(self.eod_data['strike']/self.base) * self.base)
                    self.eod_data['strike'] = self.eod_data['strike'].astype(float)
                    
        elif self.option_type == 'call':
            
            if self.strike == 'otm':
                if self.expiry == 'weekly':
                    print('getting weekly atm strikes with fullstraddle premium away')
                    print()
                    self.eod_data['strike'] = self.eod_data['close'] + self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price']
                    self.eod_data['strike'] = (round(self.eod_data['strike']/self.base) * self.base)
                    self.eod_data['strike'] = self.eod_data['strike'].astype(float)
                elif self.expiry == 'monthly':
                    print('getting monthly atm strikes with fullstraddle premium away')
                    print()
                    self.eod_data['strike'] = self.eod_data['close'] + self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price']
                    self.eod_data['strike'] = (round(self.eod_data['strike']/self.base) * self.base)
                    self.eod_data['strike'] = self.eod_data['strike'].astype(float)
            
            elif self.strike == 'itm':
                if self.expiry == 'weekly':
                    print('getting weekly otm strikes with fullstraddle premium away')
                    print()
                    self.eod_data['strike'] = self.eod_data['close'] - self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price']
                    self.eod_data['strike'] = (round(self.eod_data['strike']/self.base) * self.base)
                    self.eod_data['strike'] = self.eod_data['strike'].astype(float)
                elif self.expiry == 'monthly':
                    print('getting monthly otm strikes with fullstraddle premium away')
                    print()
                    self.eod_data['strike'] = self.eod_data['close'] - self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price']
                    self.eod_data['strike'] = (round(self.eod_data['strike']/self.base) * self.base)
                    self.eod_data['strike'] = self.eod_data['strike'].astype(float)
    
    def get_halfstraddle_strike(self):
        
        if self.option_type == 'put':

            if self.strike == 'otm':
                if self.expiry == 'weekly':
                    print('getting weekly atm strikes with halfstraddle premium away')
                    print()
                    self.eod_data['strike'] = self.eod_data['close'] - (0.5*self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price'])
                    self.eod_data['strike'] = (round(self.eod_data['strike']/self.base) * self.base)
                    self.eod_data['strike'] = self.eod_data['strike'].astype(float)
                elif self.expiry == 'monthly':
                    print('getting monthly atm strikes with halfstraddle premium away')
                    print()
                    self.eod_data['strike'] = self.eod_data['close'] - (0.5*self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price'])
                    self.eod_data['strike'] = (round(self.eod_data['strike']/self.base) * self.base)
                    self.eod_data['strike'] = self.eod_data['strike'].astype(float)
            
            elif self.strike == 'itm':
                if self.expiry == 'weekly':
                    print('getting weekly otm strikes with halfstraddle premium away')
                    print()
                    self.eod_data['strike'] = self.eod_data['close'] + (0.5*self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price'])
                    self.eod_data['strike'] = (round(self.eod_data['strike']/self.base) * self.base)
                    self.eod_data['strike'] = self.eod_data['strike'].astype(float)
                elif self.expiry == 'monthly':
                    print('getting monthly otm strikes with halfstraddle premium away')
                    print()
                    self.eod_data['strike'] = self.eod_data['close'] + (0.5*self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price'])
                    self.eod_data['strike'] = (round(self.eod_data['strike']/self.base) * self.base)
                    self.eod_data['strike'] = self.eod_data['strike'].astype(float)
                    
        elif self.option_type == 'call':
            
            if self.strike == 'otm':
                if self.expiry == 'weekly':
                    print('getting weekly atm strikes with halfstraddle premium away')
                    print()
                    self.eod_data['strike'] = self.eod_data['close'] + (0.5*self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price'])
                    self.eod_data['strike'] = (round(self.eod_data['strike']/self.base) * self.base)
                    self.eod_data['strike'] = self.eod_data['strike'].astype(float)
                elif self.expiry == 'monthly':
                    print('getting monthly atm strikes with halfstraddle premium away')
                    print()
                    self.eod_data['strike'] = self.eod_data['close'] + (0.5*self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price'])
                    self.eod_data['strike'] = (round(self.eod_data['strike']/self.base) * self.base)
                    self.eod_data['strike'] = self.eod_data['strike'].astype(float)
            
            elif self.strike == 'itm':
                if self.expiry == 'weekly':
                    print('getting weekly otm strikes with halfstraddle premium away')
                    print()
                    self.eod_data['strike'] = self.eod_data['close'] - (0.5*self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price'])
                    self.eod_data['strike'] = (round(self.eod_data['strike']/self.base) * self.base)
                    self.eod_data['strike'] = self.eod_data['strike'].astype(float)
                elif self.expiry == 'monthly':
                    print('getting monthly otm strikes with halfstraddle premium away')
                    print()
                    self.eod_data['strike'] = self.eod_data['close'] - (0.5*self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price'])
                    self.eod_data['strike'] = (round(self.eod_data['strike']/self.base) * self.base)
                    self.eod_data['strike'] = self.eod_data['strike'].astype(float)
            
    # The function that determines the strike for each day for naked strategies
    def get_naked_strike(self):

        if self.option_type == 'put':

            if self.strike == 'atm':

                # Gets the strike nearest to the close by dividing by 100 and multiplying it back by 100 and rounding off...
                #... to get to the nearest multiple of 100
                self.eod_data['strike'] = round(self.eod_data['close']/self.base) * self.base
                self.eod_data['strike'] = self.eod_data['strike'].astype(float)

            elif self.strike == 'itm':

                # Repeat the same procedure here and add up points away from atm for itm, if put
                self.eod_data['strike'] = (round(self.eod_data['close']/self.base) * self.base) + int(self.points_from_atm)
                self.eod_data['strike'] = self.eod_data['strike'].astype(float)

            else:
                # Repeat the same procedure here and subtract points away from atm for itm, if put and otm
                self.eod_data['strike'] = (round(self.eod_data['close']/self.base) * self.base) - int(self.points_from_atm)
                self.eod_data['strike'] = self.eod_data['strike'].astype(float)

        # Repeat the above procedure for call option    
        if self.option_type == 'call':

            if self.strike == 'atm':

                self.eod_data['strike'] = round(self.eod_data['close']/self.base) * self.base
                self.eod_data['strike'] = self.eod_data['strike'].astype(float)

            elif self.strike == 'itm':

                self.eod_data['strike'] = (round(self.eod_data['close']/self.base) * self.base) - int(self.points_from_atm)
                self.eod_data['strike'] = self.eod_data['strike'].astype(float)

            else:

                self.eod_data['strike'] = (round(self.eod_data['close']/self.base) * self.base) + int(self.points_from_atm)
                self.eod_data['strike'] = self.eod_data['strike'].astype(float)            
    # The function that determines the strike for each day for naked strategies            
    def get_absolute_strangle_strikes(self):
        
        print('getting the strangle strikes with absolute points')
        print()
        self.eod_data['leg_one_strike'] = (round(self.eod_data['close']/self.base) * self.base) + int(self.points_from_atm)
        self.eod_data['leg_one_strike'] = self.eod_data['leg_one_strike'].astype(float)
        self.eod_data['leg_two_strike'] = (round(self.eod_data['close']/self.base) * self.base) - int(self.points_from_atm)
        self.eod_data['leg_two_strike'] = self.eod_data['leg_two_strike'].astype(float)
    
    def get_fullstraddle_strangle_strikes(self):
        
        if self.expiry == 'weekly':
            print('getting weekly strangle strikes with fullstraddle premium away')
            print()
            self.eod_data['leg_one_strike'] = self.eod_data['close'] + self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price']
            self.eod_data['leg_one_strike'] = (round(self.eod_data['leg_one_strike']/self.base) * self.base)
            self.eod_data['leg_one_strike'] = self.eod_data['leg_one_strike'].astype(float)
            self.eod_data['leg_two_strike'] = self.eod_data['close'] - self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price']
            self.eod_data['leg_two_strike'] = (round(self.eod_data['leg_two_strike']/self.base) * self.base)
            self.eod_data['leg_two_strike'] = self.eod_data['leg_two_strike'].astype(float)
        elif self.expiry == 'monthly':
            print('getting monthly strangle strikes with fullstraddle premium away')
            print()
            self.eod_data['leg_one_strike'] = self.eod_data['close'] + self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price']
            self.eod_data['leg_one_strike'] = (round(self.eod_data['leg_one_strike']/self.base) * self.base)
            self.eod_data['leg_one_strike'] = self.eod_data['leg_one_strike'].astype(float)
            self.eod_data['leg_two_strike'] = self.eod_data['close'] - self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price']
            self.eod_data['leg_two_strike'] = (round(self.eod_data['leg_two_strike']/self.base) * self.base)
            self.eod_data['leg_two_strike'] = self.eod_data['leg_two_strike'].astype(float)
    
    def get_halfstraddle_strangle_strikes(self):
        
        if self.expiry == 'weekly':
            print('getting weekly strangle strikes with halfstraddle premium away')
            print()
            self.eod_data['leg_one_strike'] = self.eod_data['close'] + (0.5*self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price'])
            self.eod_data['leg_one_strike'] = (round(self.eod_data['leg_one_strike']/self.base) * self.base)
            self.eod_data['leg_one_strike'] = self.eod_data['leg_one_strike'].astype(float)
            self.eod_data['leg_two_strike'] = self.eod_data['close'] - (0.5*self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price'])
            self.eod_data['leg_two_strike'] = (round(self.eod_data['leg_two_strike']/self.base) * self.base)
            self.eod_data['leg_two_strike'] = self.eod_data['leg_two_strike'].astype(float)
        elif self.expiry == 'monthly':
            print('getting monthly strangle strikes with halfstraddle premium away')
            print()
            self.eod_data['leg_one_strike'] = self.eod_data['close'] + (0.5*self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price'])
            self.eod_data['leg_one_strike'] = (round(self.eod_data['leg_one_strike']/self.base) * self.base)
            self.eod_data['leg_one_strike'] = self.eod_data['leg_one_strike'].astype(float)
            self.eod_data['leg_two_strike'] = self.eod_data['close'] - (0.5*self.eod_data[f'{self.expiry}_{self.entry_time}_straddle_price'])
            self.eod_data['leg_two_strike'] = (round(self.eod_data['leg_two_strike']/self.base) * self.base)
            self.eod_data['leg_two_strike'] = self.eod_data['leg_two_strike'].astype(float)
    
    # Get the strikes for spread positions
    def get_spread_strikes(self):

        if self.option_type == 'put':

            self.eod_data['nearstrike'] = round(self.eod_data['close']/self.base) * self.base
            self.eod_data['nearstrike'] = self.eod_data['nearstrike'].astype(float)

            self.eod_data['farstrike'] = (round(self.eod_data['close']/self.base) * self.base) - int(self.points_from_atm)
            self.eod_data['farstrike'] = self.eod_data['farstrike'].astype(float)

        if self.option_type == 'call':

            self.eod_data['nearstrike'] = round(self.eod_data['close']/self.base) * self.base
            self.eod_data['nearstrike'] = self.eod_data['nearstrike'].astype(float)

            self.eod_data['farstrike'] = (round(self.eod_data['close']/self.base) * self.base) + int(self.points_from_atm)
            self.eod_data['farstrike'] = self.eod_data['farstrike'].astype(float)
            
    # This function is for combining columns to create lookup columns
    def join_columns_to_create_lookup(self, entry_columns, exit_columns, entry_column_name, exit_column_name):

        self.eod_data[entry_column_name] = self.eod_data[entry_columns].apply(lambda row: '-'.join(row.values.astype(str)), axis = 1)
        self.eod_data[exit_column_name] = self.eod_data[exit_columns].apply(lambda row: '-'.join(row.values.astype(str)), axis = 1)

    # This function is for renaming the columns to entry or exit and merging to the options_pnl dataframe to get the...
    #... close of the selected date, strike and expiry using the lookup columns
    def lookup(self, column_rename):

        self.options_data_lookup.columns = column_rename.split()

        self.options_pnl = self.options_pnl.merge(self.options_data_lookup, on = column_rename.split()[1], how = 'left')

    # This function checks for NaN values in the entry and exit columns and replaces them if found. It searches for the... 
    #... nearest traded strike for that date, expiry and strike
    def replace_nan(self, entry_column_rename, exit_column_rename, strike_column_name):

        close_entry = entry_column_rename.split()[0]
        close_exit = exit_column_rename.split()[0]
        expiry_column = 'traded_expiry'
        
        print('Entries with NaN values')
        print(self.options_pnl[self.options_pnl[close_entry].isnull()]['date'])
        print()
        
        rows_entry = self.options_pnl[self.options_pnl[close_entry].isnull()].index
        print('length of the entry rows: ', len(rows_entry))
        print()
        
        if len(rows_entry) > 1:

            for row in range(len(rows_entry) - 1):

                date, t_plus_one, strike, expiry = self.options_pnl[['date', f't+{self.holding_period}', strike_column_name,\
                                                                     expiry_column]].iloc[rows_entry[row]]
                
                test_df = self.daily_options_data[(self.daily_options_data['date'] == date) &\
                                                (self.daily_options_data['expiry'] == expiry) &\
                                                 (self.daily_options_data['option_type'] == self.option_type)]

                test_df['strike_difference'] = test_df['strike'].apply(lambda x: abs(x-strike))
                test_df = test_df.sort_values('strike_difference', ascending = True)
                
                if len(test_df) == 0:
                    print('Option data for entry missing for date: ', date)
                    print()
                    continue

                for row_count in range(len(test_df)):

                    entry = test_df['close'].iloc[row_count]
                    strike = test_df['strike'].iloc[row_count]
                    
                    exit = self.daily_options_data[(self.daily_options_data['date'] == t_plus_one) &\
                                                    (self.daily_options_data['expiry'] == expiry) &\
                                                      (self.daily_options_data['strike'] == strike) &\
                                                  (self.daily_options_data['option_type'] == self.option_type)]['close']
                    
                    if (exit.empty):
                        print('No corresponding exit data found. Iterating to the next strike')
                        print()
                        continue

                    else:
                        
                        self.options_pnl[close_entry].iloc[rows_entry[row]] = entry
                        
                        print('For date: ', date)
                        print('strike chosen is: ', strike)
                        print('and entry close is: ', self.options_pnl[close_entry].iloc[rows_entry[row]])
                        print('exit data also found. Close at exit is: ', exit.iloc[0])
                        print()
                        
                        self.options_pnl[close_exit].iloc[rows_entry[row]] = exit.iloc[0]
                        self.options_pnl[strike_column_name].iloc[rows_entry[row]] = strike
                        break
        
        print('Exits with NaN values')
        print(self.options_pnl[self.options_pnl[close_exit].isnull()]['date'])
        print()
        
        rows_exit = self.options_pnl[self.options_pnl[close_exit].isnull()].index        
        print('length of the exit rows: ', len(rows_exit))
        print()
        
        if len(rows_exit) > 1:

            for row in range(len(rows_exit) - 1):
                
                date, t_plus_one, strike, expiry = self.options_pnl[['date', f't+{self.holding_period}', strike_column_name, expiry_column]].iloc[rows_exit[row]]
                
                test_df = self.daily_options_data[(self.daily_options_data['date'] == t_plus_one) &\
                                                (self.daily_options_data['expiry'] == expiry) &\
                                                 (self.daily_options_data['option_type'] == self.option_type)]
                
                test_df['strike_difference'] = test_df['strike'].apply(lambda x: abs(x-strike))
                test_df = test_df.sort_values('strike_difference', ascending = True)
                
                if len(test_df) == 0:
                    print('Option data for exit missing for date: ', date)
                    print()
                    continue
                
                for row_count in range(len(test_df)):

                    exit = test_df['close'].iloc[row_count]
                    strike = test_df['strike'].iloc[row_count]

                    entry = self.daily_options_data[(self.daily_options_data['date'] == date) &\
                                                    (self.daily_options_data['expiry'] == expiry) &\
                                                      (self.daily_options_data['strike'] == strike)&\
                                                  (self.daily_options_data['option_type'] == self.option_type)]['close']
                    
                    if (entry.empty):
                        print('No corresponding entry data found. Iterating to the next strike')
                        print()
                        continue

                    else:

                        self.options_pnl[close_exit].iloc[rows_exit[row]] = exit
                        
                        print('For date: ', date)
                        print('strike chosen is: ', strike)
                        print('and exit close is: ', self.options_pnl[close_entry].iloc[rows_entry[row]])
                        print('entry data also found. Close at entry is: ', entry.iloc[0])
                        print()
                        self.options_pnl[close_entry].iloc[rows_exit[row]] = entry.iloc[0]
                        self.options_pnl[strike_column_name].iloc[rows_exit[row]] = strike

                        break

In [ ]:
class SignalCreator():
    
    def __init__(self, location_dict, entry_time='close', indicator_list = [], symbol = None, straddle_df = None, weekly = 1, monthly = 1, on_future=False):
           
        self.symbol = symbol
        self.weekly = weekly
        self.monthly = monthly
        self.straddle_df = straddle_df
        self.on_future = on_future
        self.entry_time = entry_time
        self.location_dict = location_dict
        
        if self.on_future:
            self.eod_data = pd.read_pickle(self.location_dict['monthly']['close'])            
        else:
            self.eod_data = pd.read_pickle(self.location_dict['weekly']['close'])
        
    def merge_entry_time_data(self):
        
        print('reading the data...')
        # Then we get the daily spot price and the daily options data for all strikes and expiries
        if self.on_future:
            entry_time_data = pd.read_pickle(self.location_dict['monthly'][self.entry_time])           
        else:
            entry_time_data = pd.read_pickle(self.location_dict['weekly'][self.entry_time])
        
        entry_time_data = entry_time_data[['date', 'close']]
        entry_time_data.columns = f'date {self.entry_time}_close'.split()
        
        self.eod_data = self.eod_data.merge(entry_time_data, on='date', how='left')
        
    def set_backtest_period(self, date=None):
        
        if date == None:
            if self.symbol == 'banknifty':
                self.eod_data = self.eod_data[self.eod_data['date'] >= datetime.date(2016, 6, 1)]
                self.eod_data.reset_index(inplace = True, drop = True)
            else:
                self.eod_data = self.eod_data[self.eod_data['date'] >= datetime.date(2019, 2, 1)]
                self.eod_data.reset_index(inplace = True, drop = True)
        else:
            self.eod_data = self.eod_data[self.eod_data['date'] >= date]
            self.eod_data.reset_index(inplace = True, drop = True)
            
    def create_ratio(self, eod_data_num=None, eod_data_denom=None):
        
        date = self.eod_data['date'].iloc[0]
        
        if pd.DataFrame(eod_data_num).empty:
            eod_data_denom = eod_data_denom[eod_data_denom['date'] >= date]
            eod_data_denom.reset_index(inplace = True, drop = True)
            self.eod_data['close_denom'] = eod_data_denom['close']
            self.eod_data['ratio'] = self.eod_data['close']/self.eod_data['close_denom']
        else:
            eod_data_num = eod_data_num[eod_data_num['date'] >= date]
            eod_data_num.reset_index(inplace = True, drop = True)
            self.eod_data['close_num'] = eod_data_num['close']
            self.eod_data['ratio'] = self.eod_data['close_num']/self.eod_data['close']
    
    def ratio_up_down(self, days=None):
        
        self.eod_data['ratio_up_down'] = np.where(self.eod_data['ratio'] >= self.eod_data['ratio'].shift(days), 'ratio-up', 'ratio-down')
        
    def get_whole_straddle_period(self):
              
        date = self.straddle_df['date'].iloc[0]
        self.eod_data = self.eod_data[self.eod_data['date'] >= date]
        self.eod_data.reset_index(inplace = True, drop = True)
            
    # Create a simple price up/down for the day based on close indicator    
    def up_down(self):
        
        self.eod_data['up/down'] = np.where(self.eod_data[f'{self.entry_time}_close'] > self.eod_data['close'].shift(), 'price-up', 'price-down')
    
    # A simple day of the week indicator
    def day_of_week(self):
        
        self.eod_data['day_of_week'] = self.eod_data['date'].apply(lambda x: x.strftime("%A"))
    
    def rsi(self, period, upper_bound = 55, lower_bound = 45):
        
        self.eod_data['change'] = self.eod_data['close'].diff()
        self.eod_data['gain'] = self.eod_data[self.eod_data['change'] >= 0]['change']
        self.eod_data['loss'] = abs(self.eod_data[self.eod_data['change'] < 0]['change'])
        self.eod_data['change'].fillna(0, inplace = True)
        self.eod_data['gain'].fillna(0, inplace = True)
        self.eod_data['loss'].fillna(0, inplace = True)
        self.eod_data['average_gain'] = self.eod_data['gain'].rolling(window = period).mean()
        self.eod_data['average_gain'].fillna(0, inplace = True)
        self.eod_data['average_loss'] = self.eod_data['loss'].rolling(window = period).mean()
        self.eod_data['average_loss'].fillna(0, inplace = True)
        self.eod_data['average_gain_rsi'] = 0
        self.eod_data['average_loss_rsi'] = 0

        try:
            for _ in range(period, len(self.eod_data)):

                self.eod_data['average_gain_rsi'].iloc[_] = ((self.eod_data['average_gain'].iloc[_ - 1] * (period - 1)) +\
                                                                 self.eod_data['gain'].iloc[_])/(14)
                self.eod_data['average_loss_rsi'].iloc[_] = ((self.eod_data['average_loss'].iloc[_ - 1] * (period - 1)) +\
                                                                 self.eod_data['loss'].iloc[_])/(14)        

            self.eod_data['rs'] = self.eod_data['average_gain_rsi']/self.eod_data['average_loss_rsi']
            self.eod_data['rsi'] = (100 - (100/(1 + self.eod_data['rs'])))

            self.eod_data.drop(['change', 'gain', 'loss', 'average_gain', 'average_loss', 'average_gain_rsi', 'average_loss_rsi', 'rs'], axis = 1, inplace = True)

        except Exception as e:

            print('Error while calculation rsi is:', e)
        
        self.eod_data['rsi'] = np.where((self.eod_data['rsi'] > lower_bound) & (self.eod_data['rsi'] < upper_bound), 'neutral-rsi',\
                             np.where(self.eod_data['rsi'] >= upper_bound, 'bullish-rsi', 'bearish-rsi'))
    
    def adx(self, period, trend_bound = 25):
        
        self.eod_data['h-l'] = self.eod_data['high'] - self.eod_data['low']
        self.eod_data['h-prev_close'] = self.eod_data['high'] - self.eod_data['close'].shift()
        self.eod_data['prev_close-l'] = self.eod_data['close'].shift() - self.eod_data['close']

        self.eod_data['h-prev_close'] = self.eod_data['h-prev_close'].apply(lambda x: abs(x))
        self.eod_data['prev_close-l'] = self.eod_data['prev_close-l'].apply(lambda x: abs(x))

        self.eod_data['true_range'] = self.eod_data[['h-l', 'h-prev_close', 'prev_close-l']].max(axis = 1)

        self.eod_data.drop(['h-l', 'h-prev_close', 'prev_close-l'], axis = 1, inplace = True)

        self.eod_data['positive_dm'] = np.where((self.eod_data['high'] - self.eod_data['high'].shift()) > (self.eod_data['low'].shift() - self.eod_data['low']),\
                                     self.eod_data['high'] - self.eod_data['high'].shift(), 0)
        self.eod_data['positive_dm'] = self.eod_data['positive_dm'].apply(lambda x: max(x, 0))
        self.eod_data['negative_dm'] = np.where((self.eod_data['high'] - self.eod_data['high'].shift()) < (self.eod_data['low'].shift() - self.eod_data['low']),\
                                             self.eod_data['low'].shift() - self.eod_data['low'], 0)
        self.eod_data['negative_dm'] = self.eod_data['negative_dm'].apply(lambda x: max(x, 0))
        
        self.eod_data['true_range_14'] = self.eod_data['true_range'].rolling(window = 14).sum()
        
        self.eod_data['true_range_14'] = self.eod_data['true_range'].rolling(window = 14).sum()
        self.eod_data['plus_dmi_14'] = self.eod_data['positive_dm'].rolling(window = 14).sum()
        self.eod_data['minus_dmi_14'] = self.eod_data['negative_dm'].rolling(window = 14).sum()

        try:
            for _ in range(15, len(self.eod_data)):

                self.eod_data['true_range_14'].iloc[_] = self.eod_data['true_range_14'].iloc[_ - 1] - (self.eod_data['true_range_14'].iloc[_ - 1]/14)\
                                                      + self.eod_data['true_range'].iloc[_]

                self.eod_data['plus_dmi_14'].iloc[_] = self.eod_data['plus_dmi_14'].iloc[_ - 1] - (self.eod_data['plus_dmi_14'].iloc[_ - 1]/14)\
                                                      + self.eod_data['plus_dmi_14'].iloc[_]
                self.eod_data['minus_dmi_14'].iloc[_] = self.eod_data['minus_dmi_14'].iloc[_ - 1] - (self.eod_data['minus_dmi_14'].iloc[_ - 1]/14)\
                                                      + self.eod_data['minus_dmi_14'].iloc[_]        

            self.eod_data['plus_di_14'] = 100 * (self.eod_data['plus_dmi_14']/self.eod_data['true_range_14'])
            self.eod_data['minus_di_14'] = 100 * (self.eod_data['minus_dmi_14']/self.eod_data['true_range_14'])

            self.eod_data['diff_di'] = abs(self.eod_data['plus_di_14'] - self.eod_data['minus_di_14'])
            self.eod_data['sum_di'] = self.eod_data['plus_di_14'] + self.eod_data['minus_di_14']

            self.eod_data['dx'] = 100 * (self.eod_data['diff_di']/self.eod_data['sum_di'])
            self.eod_data['adx'] = self.eod_data['dx'].rolling(window = 14).mean()

            self.eod_data.drop(['true_range_14', 'positive_dm', 'negative_dm', 'plus_dmi_14', 'minus_dmi_14',\
                             'diff_di', 'sum_di', 'dx', 'true_range', 'plus_di_14', 'minus_di_14'], axis = 1, inplace = True)

        except Exception as e:

            print('Error while calculation adx is:', e)

        self.eod_data.dropna(inplace = True)
        
        self.eod_data['adx'] = np.where((self.eod_data['adx'] > 25) & (self.eod_data['adx'] > self.eod_data['adx'].shift()),\
                                        'trending-adx', 'neutral-adx')
        
    def days_remaining_in_month(self):
        
        self.eod_data['days_remain_month'] = self.eod_data['date'].apply(lambda x: calendar.monthrange(x.year, x.month)[1] - x.day)
    
    def get_business_days_remaining_month(self):
    
        self.eod_data['day_of_month'] = 0
        for _ in range(1, len(self.eod_data)):

            if self.eod_data['date'].iloc[_].month != self.eod_data['date'].iloc[_ - 1].month:
                self.eod_data['day_of_month'].iloc[_] = 1

            else:
                self.eod_data['day_of_month'].iloc[_] = 1 + self.eod_data['day_of_month'].iloc[_ - 1]

        self.eod_data['business_days_remaining_in_month'] = np.where(self.eod_data['day_of_month'] > self.eod_data['day_of_month'].shift(-1), self.eod_data['day_of_month'], np.nan)

        self.eod_data['business_days_remaining_in_month'].bfill(inplace = True)

        self.eod_data = self.eod_data[~self.eod_data['business_days_remaining_in_month'].isnull()]

        self.eod_data['business_days_remaining_in_month'] = self.eod_data['business_days_remaining_in_month'] - self.eod_data['day_of_month']
        
    def moving_average(self, period = '70'):
        
        self.eod_data[f'ma_{period}'] = self.eod_data['close'].rolling(window = int(period)).mean()
        self.eod_data[f'ma_{period}'] = self.eod_data['close']/self.eod_data[f'ma_{period}']
        self.eod_data[f'ma_{period}'] = np.where(self.eod_data[f'ma_{period}'] > 1, f'above_ma_{period}', f'below_ma_{period}')
    
    def full_gap(self):
        
        self.eod_data['full_gap'] = np.where(self.eod_data['open'] >= self.eod_data['high'].shift(), 'fgu',\
                                            np.where(self.eod_data['open'] <= self.eod_data['low'].shift(), 'fgd', '-'))
    
    def gap(self):
        
        self.eod_data['gap'] = np.where(self.eod_data['open'] >= self.eod_data['close'].shift(), 'gu',\
                                       np.where(self.eod_data['open'] <= self.eod_data['close'].shift(), 'gd', '-'))
    
    def low_greater_close(self):
        
        self.eod_data['low>close'] = np.where(self.eod_data['low'] >= self.eod_data['close'].shift(), 'low>close', '-')
        
    def get_monthly_iv(self):
        
        straddle_monthly = self.straddle_df[['date', 'monthly_close_iv', f'monthly_{self.entry_time}_iv']]
        self.eod_data = self.eod_data.merge(straddle_monthly, on = 'date', how = 'left')
    
    def get_weekly_iv(self):
        
        straddle_weekly = self.straddle_df[['date', 'weekly_iv']]
        self.eod_data = self.eod_data.merge(straddle_weekly, on = 'date', how = 'left')
    
    def funcprint(self, x):
        x['iv'] = np.where(x['weekly_iv']>x['weekly_iv'].shift(), 'ivup', 'ivdown')
        return x
    
    def get_iv_dow_up_down(self):
        self.eod_data['iv_day_of_week'] = pd.DataFrame(self.eod_data.groupby('day_of_week')[['date','weekly_iv']].apply(self.funcprint))['iv']
    
    def close_below_above_std(self, window=20, multiplier=1):    
        self.eod_data[f'std_{window}'] = np.log(self.eod_data['close']/self.eod_data['close'].shift()).rolling(window = window).std()
        self.eod_data[f'close_std_{window}'] = np.where(self.eod_data['close'] < (self.eod_data['close'].shift() * (1-(multiplier*self.eod_data[f'std_{window}'].shift()))), f'bel_{multiplier}_std_close', np.where(self.eod_data['close'] > (self.eod_data['close'].shift() * (1+(multiplier*self.eod_data[f'std_{window}'].shift()))), f'abv_{multiplier}_std_close', '-'))
        
    def log_volatility(self, window = 20):
        
        self.eod_data[f'std_{window}'] = np.log(self.eod_data['close']/self.eod_data['close'].shift()).rolling(window = window).std() * math.sqrt(250) * 100       
    
    def mad(self, window = 20):
        
        self.eod_data[f'mad_{window}'] = (((self.eod_data['close'] * 100)/self.eod_data['close'].shift()) - 100).rolling(window = window).apply(lambda x: round(x.mad() * 19.82, 2))
        
    def parkinson_volatility(self, window = 20):
        
        self.eod_data[f'parkinson_vol_{window}'] = np.sqrt(((np.log(self.eod_data['high']/self.eod_data['low']) ** 2).\
                                                  rolling(window).sum()/4/window/np.log(2))) * np.sqrt(250) * 100
    
    def yang_zhang(self, window = 20):
        
        log_ho = (self.eod_data['high'] / self.eod_data['open']).apply(np.log)
        log_lo = (self.eod_data['low'] / self.eod_data['open']).apply(np.log)
        log_co = (self.eod_data['close'] / self.eod_data['open']).apply(np.log)

        log_oc = (self.eod_data['open'] / self.eod_data['close'].shift(1)).apply(np.log)
        log_oc_sq = log_oc**2

        log_cc = (self.eod_data['close'] / self.eod_data['close'].shift(1)).apply(np.log)
        log_cc_sq = log_cc**2

        rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)

        close_vol = log_cc_sq.rolling(window=window, center=False).sum() * (1.0 / (window - 1.0))
        open_vol = log_oc_sq.rolling(window=window, center=False).sum() * (1.0 / (window - 1.0))
        window_rs = rs.rolling(window=window, center=False).sum() * (1.0 / (window - 1.0))

        k = 0.34 / (1.34 + (window + 1) / (window - 1))
        result = (open_vol + k * close_vol + (1 - k) * window_rs).apply(np.sqrt) * math.sqrt(250)
        self.eod_data[f'yang_zhang_vol_{window}'] = result * 100
    
    def garman_klass(self, window = 20):
        
        log_hl = (self.eod_data['high'] / self.eod_data['low']).apply(np.log)
        log_co = (self.eod_data['close'] / self.eod_data['open']).apply(np.log)
        rs = 0.5 * log_hl**2 - (2*math.log(2)-1) * log_co**2
        result = rs.rolling(window=window, center=False).apply(lambda x: (x.mean() * 250)**0.5)
        self.eod_data[f'garman_klass_vol_{window}'] = result * 100
    
    def rogers_satchell(self, window = 20):
        
        log_ho = (self.eod_data['high'] / self.eod_data['open']).apply(np.log)
        log_lo = (price_data['low'] / self.eod_data['open']).apply(np.log)
        log_co = (self.eod_data['close'] / self.eod_data['open']).apply(np.log)
    
        rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)
        result = rs.rolling(window=window, center=False).apply(lambda x: (x.mean() * 250)**0.5)
        self.eod_data[f'garman_klass_vol_{window}'] = result * 100
    
    def opex_minus_one(self):
       
        self.eod_data = self.eod_data[self.eod_data['t+1'] == self.eod_data['current_week_expiry']]
    
    def get_opex_column(self):

        conditions = [self.eod_data['t+1'].shift() == self.eod_data['current_week_expiry'].shift(),
                     self.eod_data['t+1'] == self.eod_data['current_week_expiry'],
                     self.eod_data['t+1'].shift(-1) == self.eod_data['current_week_expiry'].shift(-1),
                     (self.eod_data['t+1'].shift(-2) == self.eod_data['current_week_expiry'].shift(-2)) & (self.eod_data['t+1'].shift() != self.eod_data['current_week_expiry'].shift()),
                     (self.eod_data['t+1'].shift(-3) == self.eod_data['current_week_expiry'].shift(-3)) & (self.eod_data['t+1'].shift() != self.eod_data['current_week_expiry'].shift())]

        choices = ['opex', 'opex1', 'opex2', 'opex3', 'opex4']
        
        self.eod_data['opex'] = np.select(conditions, choices)

    def high_close(self, period=20):
        
        self.eod_data[f'close_high_{period}'] = self.eod_data['close'].rolling(window = period).max()
    
    def high_close_indicator(self, period=20):
        
        self.high_close(period=period)
        self.eod_data[f'close_high_{period}'] = np.where(self.eod_data[f'{entry_time}_close'] >= self.eod_data[f'close_high_{period}'].shift(),\
                                                         f'{period}Dhigh', '-')
        
    def low_close(self, period=20):
        
        self.eod_data[f'close_low_{period}'] = self.eod_data['close'].rolling(window = period).min()
    
    def low_close_indicator(self, period=20):
        
        self.low_close(period=period)
        self.eod_data[f'close_low_{period}'] = np.where(self.eod_data[f'{entry_time}_close'] <= self.eod_data[f'close_low_{period}'].shift(),\
                                                         f'{period}Dlow', '-')
        
    def high_low_cycle(self, period = 20):
        
        self.eod_data[f'{period}_high_low_cycle'] = np.where(self.eod_data[f'close_high_{period}'] == f'{period}Dhigh', f'{period}Dhigh', np.where(self.eod_data[f'close_low_{period}'] == f'{period}Dlow', f'{period}Dlow', np.nan))
        self.eod_data[f'{period}_high_low_cycle'] = self.eod_data[f'{period}_high_low_cycle'].mask(eod_indicator_df.eod_data[f'{period}_high_low_cycle']=='nan', None).ffill()
    
    def narrow_range(self, period = 5):
        
        self.eod_data[f'nr{period}'] = np.where(abs(np.log(self.eod_data['close']/self.eod_data['close'].shift())).rolling(window = period).min() == abs(np.log(self.eod_data['close']/self.eod_data['close'].shift())), f'nr{period}', '-')
        
    def max_min_percent_move(self, period=10):
        
        self.eod_data[f'max_percent_move_{period}'] = ((self.eod_data['close'] - self.eod_data['close'].shift())/self.eod_data['close'].shift()).rolling(window = period).max()
        self.eod_data[f'min_percent_move_{period}'] = ((self.eod_data['close'] - self.eod_data['close'].shift())/self.eod_data['close'].shift()).rolling(window = period).min()
        
    def max_min_move(self, period = 10):
        
        self.max_min_percent_move(period=period)
        self.eod_data[f'max_min_move_{period}_days'] = np.where(((self.eod_data[f'{entry_time}_close'] - self.eod_data['close'].shift())/self.eod_data['close'].shift()) >= (self.eod_data[f'max_percent_move_{period}'].shift()), 'max-move', np.where(((self.eod_data[f'{entry_time}_close'] - self.eod_data['close'].shift())/self.eod_data['close'].shift()) <= (self.eod_data[f'min_percent_move_{period}'].shift()), 'min-move', '-'))

    def first_20D_high_low_close_expiry(self, period = 20):
        
        self.get_opex_column()
        self.high_close(period = period)
        self.low_close(period = period)
        
        conditions = [(self.eod_data[f'close_high_{period}'] == f'{period}Dhigh') & (self.eod_data['opex'] == 'opex4'),
                      (self.eod_data[f'close_high_{period}'] == f'{period}Dhigh') & (self.eod_data['opex'] == 'opex3') & (self.eod_data['opex'].shift() != 'opex4'),
                      (self.eod_data[f'close_high_{period}'] == f'{period}Dhigh') & (self.eod_data['opex'] == 'opex3') & (self.eod_data['opex'].shift() == 'opex4') & (self.eod_data[f'close_high_{period}'].shift() != f'{period}Dhigh'),
                      (self.eod_data[f'close_high_{period}'] == f'{period}Dhigh') & (self.eod_data['opex'] == 'opex2') & (self.eod_data['opex'].shift() != 'opex3'),
                      (self.eod_data[f'close_high_{period}'] == f'{period}Dhigh') & (self.eod_data['opex'] == 'opex2') & (self.eod_data['opex'].shift() == 'opex3') & (self.eod_data[f'close_high_{period}'].shift() != f'{period}Dhigh') & (self.eod_data['opex'].shift(2) != 'opex4'),
                      (self.eod_data[f'close_high_{period}'] == f'{period}Dhigh') & (self.eod_data['opex'] == 'opex2') & (self.eod_data['opex'].shift() == 'opex3') & (self.eod_data[f'close_high_{period}'].shift() != f'{period}Dhigh') & (self.eod_data['opex'].shift(2) == 'opex4') & (self.eod_data[f'close_high_{period}'].shift(2) != f'{period}Dhigh'),
                      (self.eod_data[f'close_high_{period}'] == f'{period}Dhigh') & (self.eod_data['opex'] == 'opex1') & (self.eod_data['opex'].shift() != 'opex2'),
                      (self.eod_data[f'close_high_{period}'] == f'{period}Dhigh') & (self.eod_data['opex'] == 'opex1') & (self.eod_data['opex'].shift() == 'opex2') & (self.eod_data[f'close_high_{period}'].shift() != f'{period}Dhigh') & (self.eod_data['opex'].shift(2) != 'opex3'),
                      (self.eod_data[f'close_high_{period}'] == f'{period}Dhigh') & (self.eod_data['opex'] == 'opex1') & (self.eod_data['opex'].shift() == 'opex2') & (self.eod_data[f'close_high_{period}'].shift() != f'{period}Dhigh') & (self.eod_data['opex'].shift(2) == 'opex3') & (self.eod_data[f'close_high_{period}'].shift(2) != f'{period}Dhigh') & (self.eod_data['opex'].shift(3) != 'opex4'),
                      (self.eod_data[f'close_high_{period}'] == f'{period}Dhigh') & (self.eod_data['opex'] == 'opex1') & (self.eod_data['opex'].shift() == 'opex2') & (self.eod_data[f'close_high_{period}'].shift() != f'{period}Dhigh') & (self.eod_data['opex'].shift(2) == 'opex3') & (self.eod_data[f'close_high_{period}'].shift(2) != f'{period}Dhigh') & (self.eod_data['opex'].shift(3) == 'opex4') & (self.eod_data[f'close_high_{period}'].shift(3) != f'{period}Dhigh'),
                      (self.eod_data[f'close_high_{period}'] == f'{period}Dhigh') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() != 'opex1'),
                      (self.eod_data[f'close_high_{period}'] == f'{period}Dhigh') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() == 'opex1') & (self.eod_data[f'close_high_{period}'].shift() != f'{period}Dhigh') & (self.eod_data['opex'].shift(2) != 'opex2'),
                      (self.eod_data[f'close_high_{period}'] == f'{period}Dhigh') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() == 'opex1') & (self.eod_data[f'close_high_{period}'].shift() != f'{period}Dhigh') & (self.eod_data['opex'].shift(2) == 'opex2') & (self.eod_data[f'close_high_{period}'].shift(2) != f'{period}Dhigh') & (self.eod_data['opex'].shift(3) != 'opex3'),
                      (self.eod_data[f'close_high_{period}'] == f'{period}Dhigh') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() == 'opex1') & (self.eod_data[f'close_high_{period}'].shift() != f'{period}Dhigh') & (self.eod_data['opex'].shift(2) == 'opex2') & (self.eod_data[f'close_high_{period}'].shift(2) != f'{period}Dhigh') & (self.eod_data['opex'].shift(3) == 'opex3') & (self.eod_data[f'close_high_{period}'].shift(3) != f'{period}Dhigh') & (self.eod_data['opex'].shift(4) != 'opex4'),
                      (self.eod_data[f'close_high_{period}'] == f'{period}Dhigh') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() == 'opex1') & (self.eod_data[f'close_high_{period}'].shift() != f'{period}Dhigh') & (self.eod_data['opex'].shift(2) == 'opex2') & (self.eod_data[f'close_high_{period}'].shift(2) != f'{period}Dhigh') & (self.eod_data['opex'].shift(3) == 'opex3') & (self.eod_data[f'close_high_{period}'].shift(3) != f'{period}Dhigh') & (self.eod_data['opex'].shift(4) == 'opex4') & (self.eod_data[f'close_high_{period}'].shift(4) != f'{period}Dhigh'),
                      (self.eod_data[f'close_low_{period}'] == f'{period}Dlow') & (self.eod_data['opex'] == 'opex4'),
                      (self.eod_data[f'close_low_{period}'] == f'{period}Dlow') & (self.eod_data['opex'] == 'opex3') & (self.eod_data['opex'].shift() != 'opex4'),
                      (self.eod_data[f'close_low_{period}'] == f'{period}Dlow') & (self.eod_data['opex'] == 'opex3') & (self.eod_data['opex'].shift() == 'opex4') & (self.eod_data[f'close_low_{period}'].shift() != f'{period}Dlow'),
                      (self.eod_data[f'close_low_{period}'] == f'{period}Dlow') & (self.eod_data['opex'] == 'opex2') & (self.eod_data['opex'].shift() != 'opex3'),
                      (self.eod_data[f'close_low_{period}'] == f'{period}Dlow') & (self.eod_data['opex'] == 'opex2') & (self.eod_data['opex'].shift() == 'opex3') & (self.eod_data[f'close_low_{period}'].shift() != f'{period}Dlow') & (self.eod_data['opex'].shift(2) != 'opex4'),
                      (self.eod_data[f'close_low_{period}'] == f'{period}Dlow') & (self.eod_data['opex'] == 'opex2') & (self.eod_data['opex'].shift() == 'opex3') & (self.eod_data[f'close_low_{period}'].shift() != f'{period}Dlow') & (self.eod_data['opex'].shift(2) == 'opex4') & (self.eod_data[f'close_low_{period}'].shift(2) != f'{period}Dlow'),
                      (self.eod_data[f'close_low_{period}'] == f'{period}Dlow') & (self.eod_data['opex'] == 'opex1') & (self.eod_data['opex'].shift() != 'opex2'),
                      (self.eod_data[f'close_low_{period}'] == f'{period}Dlow') & (self.eod_data['opex'] == 'opex1') & (self.eod_data['opex'].shift() == 'opex2') & (self.eod_data[f'close_low_{period}'].shift() != f'{period}Dlow') & (self.eod_data['opex'].shift(2) != 'opex3'),
                      (self.eod_data[f'close_low_{period}'] == f'{period}Dlow') & (self.eod_data['opex'] == 'opex1') & (self.eod_data['opex'].shift() == 'opex2') & (self.eod_data[f'close_low_{period}'].shift() != f'{period}Dlow') & (self.eod_data['opex'].shift(2) == 'opex3') & (self.eod_data[f'close_low_{period}'].shift(2) != f'{period}Dlow') & (self.eod_data['opex'].shift(3) != 'opex4'),
                      (self.eod_data[f'close_low_{period}'] == f'{period}Dlow') & (self.eod_data['opex'] == 'opex1') & (self.eod_data['opex'].shift() == 'opex2') & (self.eod_data[f'close_low_{period}'].shift() != f'{period}Dlow') & (self.eod_data['opex'].shift(2) == 'opex3') & (self.eod_data[f'close_low_{period}'].shift(2) != f'{period}Dlow') & (self.eod_data['opex'].shift(3) == 'opex4') & (self.eod_data[f'close_low_{period}'].shift(3) != f'{period}Dlow'),
                      (self.eod_data[f'close_low_{period}'] == f'{period}Dlow') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() != 'opex1'),
                      (self.eod_data[f'close_low_{period}'] == f'{period}Dlow') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() == 'opex1') & (self.eod_data[f'close_low_{period}'].shift() != f'{period}Dlow') & (self.eod_data['opex'].shift(2) != 'opex2'),
                      (self.eod_data[f'close_low_{period}'] == f'{period}Dlow') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() == 'opex1') & (self.eod_data[f'close_low_{period}'].shift() != f'{period}Dlow') & (self.eod_data['opex'].shift(2) == 'opex2') & (self.eod_data[f'close_low_{period}'].shift(2) != f'{period}Dlow') & (self.eod_data['opex'].shift(3) != 'opex3'),
                      (self.eod_data[f'close_low_{period}'] == f'{period}Dlow') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() == 'opex1') & (self.eod_data[f'close_low_{period}'].shift() != f'{period}Dlow') & (self.eod_data['opex'].shift(2) == 'opex2') & (self.eod_data[f'close_low_{period}'].shift(2) != f'{period}Dlow') & (self.eod_data['opex'].shift(3) == 'opex3') & (self.eod_data[f'close_low_{period}'].shift(3) != f'{period}Dlow') & (self.eod_data['opex'].shift(4) != 'opex4'),
                      (self.eod_data[f'close_low_{period}'] == f'{period}Dlow') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() == 'opex1') & (self.eod_data[f'close_low_{period}'].shift() != f'{period}Dlow') & (self.eod_data['opex'].shift(2) == 'opex2') & (self.eod_data[f'close_low_{period}'].shift(2) != f'{period}Dlow') & (self.eod_data['opex'].shift(3) == 'opex3') & (self.eod_data[f'close_low_{period}'].shift(3) != f'{period}Dlow') & (self.eod_data['opex'].shift(4) == 'opex4') & (self.eod_data[f'close_low_{period}'].shift(4) != f'{period}Dlow')]

        choices = 15 * [f'{period}Dhigh'] + 15 * [f'{period}Dlow']

        self.eod_data[f'first-{period}D-high-low'] = np.select(conditions, choices)
    
    def first_max_min_move_expiry(self, period = 10):
        
        self.get_opex_column()
        self.max_min_move(period = period)
        
        conditions = [(self.eod_data[f'max_min_move_{period}_days'] == 'max-move') & (self.eod_data['opex'] == 'opex4'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'max-move') & (self.eod_data['opex'] == 'opex3') & (self.eod_data['opex'].shift() != 'opex4'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'max-move') & (self.eod_data['opex'] == 'opex3') & (self.eod_data['opex'].shift() == 'opex4') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'max-move'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'max-move') & (self.eod_data['opex'] == 'opex2') & (self.eod_data['opex'].shift() != 'opex3'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'max-move') & (self.eod_data['opex'] == 'opex2') & (self.eod_data['opex'].shift() == 'opex3') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'max-move') & (self.eod_data['opex'].shift(2) != 'opex4'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'max-move') & (self.eod_data['opex'] == 'opex2') & (self.eod_data['opex'].shift() == 'opex3') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'max-move') & (self.eod_data['opex'].shift(2) == 'opex4') & (self.eod_data[f'max_min_move_{period}_days'].shift(2) != 'max-move'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'max-move') & (self.eod_data['opex'] == 'opex1') & (self.eod_data['opex'].shift() != 'opex2'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'max-move') & (self.eod_data['opex'] == 'opex1') & (self.eod_data['opex'].shift() == 'opex2') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'max-move') & (self.eod_data['opex'].shift(2) != 'opex3'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'max-move') & (self.eod_data['opex'] == 'opex1') & (self.eod_data['opex'].shift() == 'opex2') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'max-move') & (self.eod_data['opex'].shift(2) == 'opex3') & (self.eod_data[f'max_min_move_{period}_days'].shift(2) != 'max-move') & (self.eod_data['opex'].shift(3) != 'opex4'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'max-move') & (self.eod_data['opex'] == 'opex1') & (self.eod_data['opex'].shift() == 'opex2') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'max-move') & (self.eod_data['opex'].shift(2) == 'opex3') & (self.eod_data[f'max_min_move_{period}_days'].shift(2) != 'max-move') & (self.eod_data['opex'].shift(3) == 'opex4') & (self.eod_data[f'max_min_move_{period}_days'].shift(3) != 'max-move'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'max-move') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() != 'opex1'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'max-move') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() == 'opex1') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'max-move') & (self.eod_data['opex'].shift(2) != 'opex2'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'max-move') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() == 'opex1') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'max-move') & (self.eod_data['opex'].shift(2) == 'opex2') & (self.eod_data[f'max_min_move_{period}_days'].shift(2) != 'max-move') & (self.eod_data['opex'].shift(3) != 'opex3'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'max-move') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() == 'opex1') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'max-move') & (self.eod_data['opex'].shift(2) == 'opex2') & (self.eod_data[f'max_min_move_{period}_days'].shift(2) != 'max-move') & (self.eod_data['opex'].shift(3) == 'opex3') & (self.eod_data[f'max_min_move_{period}_days'].shift(3) != 'max-move') & (self.eod_data['opex'].shift(4) != 'opex4'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'max-move') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() == 'opex1') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'max-move') & (self.eod_data['opex'].shift(2) == 'opex2') & (self.eod_data[f'max_min_move_{period}_days'].shift(2) != 'max-move') & (self.eod_data['opex'].shift(3) == 'opex3') & (self.eod_data[f'max_min_move_{period}_days'].shift(3) != 'max-move') & (self.eod_data['opex'].shift(4) == 'opex4') & (self.eod_data[f'max_min_move_{period}_days'].shift(4) != 'max-move'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'min-move') & (self.eod_data['opex'] == 'opex4'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'min-move') & (self.eod_data['opex'] == 'opex3') & (self.eod_data['opex'].shift() != 'opex4'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'min-move') & (self.eod_data['opex'] == 'opex3') & (self.eod_data['opex'].shift() == 'opex4') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'min-move'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'min-move') & (self.eod_data['opex'] == 'opex2') & (self.eod_data['opex'].shift() != 'opex3'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'min-move') & (self.eod_data['opex'] == 'opex2') & (self.eod_data['opex'].shift() == 'opex3') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'min-move') & (self.eod_data['opex'].shift(2) != 'opex4'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'min-move') & (self.eod_data['opex'] == 'opex2') & (self.eod_data['opex'].shift() == 'opex3') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'min-move') & (self.eod_data['opex'].shift(2) == 'opex4') & (self.eod_data[f'max_min_move_{period}_days'].shift(2) != 'min-move'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'min-move') & (self.eod_data['opex'] == 'opex1') & (self.eod_data['opex'].shift() != 'opex2'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'min-move') & (self.eod_data['opex'] == 'opex1') & (self.eod_data['opex'].shift() == 'opex2') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'min-move') & (self.eod_data['opex'].shift(2) != 'opex3'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'min-move') & (self.eod_data['opex'] == 'opex1') & (self.eod_data['opex'].shift() == 'opex2') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'min-move') & (self.eod_data['opex'].shift(2) == 'opex3') & (self.eod_data[f'max_min_move_{period}_days'].shift(2) != 'min-move') & (self.eod_data['opex'].shift(3) != 'opex4'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'min-move') & (self.eod_data['opex'] == 'opex1') & (self.eod_data['opex'].shift() == 'opex2') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'min-move') & (self.eod_data['opex'].shift(2) == 'opex3') & (self.eod_data[f'max_min_move_{period}_days'].shift(2) != 'min-move') & (self.eod_data['opex'].shift(3) == 'opex4') & (self.eod_data[f'max_min_move_{period}_days'].shift(3) != 'min-move'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'min-move') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() != 'opex1'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'min-move') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() == 'opex1') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'min-move') & (self.eod_data['opex'].shift(2) != 'opex2'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'min-move') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() == 'opex1') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'min-move') & (self.eod_data['opex'].shift(2) == 'opex2') & (self.eod_data[f'max_min_move_{period}_days'].shift(2) != 'min-move') & (self.eod_data['opex'].shift(3) != 'opex3'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'min-move') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() == 'opex1') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'min-move') & (self.eod_data['opex'].shift(2) == 'opex2') & (self.eod_data[f'max_min_move_{period}_days'].shift(2) != 'min-move') & (self.eod_data['opex'].shift(3) == 'opex3') & (self.eod_data[f'max_min_move_{period}_days'].shift(3) != 'min-move') & (self.eod_data['opex'].shift(4) != 'opex4'),
                      (self.eod_data[f'max_min_move_{period}_days'] == 'min-move') & (self.eod_data['opex'] == 'opex') & (self.eod_data['opex'].shift() == 'opex1') & (self.eod_data[f'max_min_move_{period}_days'].shift() != 'min-move') & (self.eod_data['opex'].shift(2) == 'opex2') & (self.eod_data[f'max_min_move_{period}_days'].shift(2) != 'min-move') & (self.eod_data['opex'].shift(3) == 'opex3') & (self.eod_data[f'max_min_move_{period}_days'].shift(3) != 'min-move') & (self.eod_data['opex'].shift(4) == 'opex4') & (self.eod_data[f'max_min_move_{period}_days'].shift(4) != 'min-move')]

        choices = 15 * ['first-max-move'] + 15 * ['first-min-move']

        self.eod_data[f'first-max-min-move-{period}'] = np.select(conditions, choices)

In [1]:
# df_list has columns (entry_date, exit_date, pnl, strategy_name)

class Backtest:
    
    def __init__(self, df_list=None, initial_capital=1000000, quantity=1, lot_size=25):
        
        self.df_list = df_list
        self.starting_capital = initial_capital
        self.quantity = quantity
        self.lot_size = lot_size
    
    def get_daily_dates(self):
        
        self.portfolio_trade_log = pd.DataFrame(pd.date_range(
          start=str(self.start_date), end=str(self.end_date)), columns=['date'])
        self.portfolio_trade_log['date'] = self.portfolio_trade_log['date'].apply(lambda x: x.date())
        self.portfolio_trade_log.set_index('date', inplace=True)
        self.portfolio_trade_log['overnight_short_position'] = 0
        self.portfolio_trade_log['intraday_short_position'] = 0
        self.portfolio_trade_log['overnight_long_position'] = 0
        self.portfolio_trade_log['intraday_long_position'] = 0
    
    def add_strategy_positions_to_trade_log(self):
        
        for df in self.df_list:
    
            strategy_name = df['strategy_name'].unique()[0]
            self.portfolio_trade_log[f'pnl-{strategy_name}'] = 0.0
            self.portfolio_trade_log[f'position-{strategy_name}'] = 0        
    
    def get_portfolio_trade_log(self):
        
        for date in self.portfolio_trade_log.index:
    
            positions  = 0
            combined_pnl = 0

            for df in self.df_list:

                strategy_name = df['strategy_name'].unique()[0]
                
                try:
                    entry_date = df[df['entry_date'] == date]['entry_date'].iloc[0]
                    self.portfolio_trade_log.at[date, f'position-{strategy_name}'] = 1
                    if strategy_name.split('-')[0] == 'short':
                        self.portfolio_trade_log.at[date, 'overnight_short_position'] = -1 + self.portfolio_trade_log.at[date, 'overnight_short_position']
                    else:
                        self.portfolio_trade_log.at[date, 'overnight_long_position'] = 1 + self.portfolio_trade_log.at[date, 'overnight_long_position']
                except:
                    pass
                
                temp = df[(df['entry_date'] < date) & (df['exit_date'] > date)]
                if (len(temp) > 0):
                    
                    self.portfolio_trade_log.at[date, f'position-{strategy_name}'] = 1
                    
                    if strategy_name.split('-')[0] == 'short':
                        self.portfolio_trade_log.at[date, 'overnight_short_position'] = -1 + self.portfolio_trade_log.at[date, 'overnight_short_position']
                        self.portfolio_trade_log.at[date, 'intraday_short_position'] = -1 + self.portfolio_trade_log.at[date, 'intraday_short_position']
                    else:
                        self.portfolio_trade_log.at[date, 'overnight_long_position'] = 1 + self.portfolio_trade_log.at[date, 'overnight_long_position']
                        self.portfolio_trade_log.at[date, 'intraday_long_position'] = 1 + self.portfolio_trade_log.at[date, 'intraday_long_position']
                else:
                    pass

                try:
                    exit_date = df[df['exit_date'] == date]['exit_date'].iloc[0]
                    pnl = df[df['exit_date'] == date]['pnl'].iloc[0]
                    self.portfolio_trade_log.at[date, f'pnl-{strategy_name}'] = pnl
                    
                    if strategy_name.split('-')[0] == 'short':
                        self.portfolio_trade_log.at[date, 'intraday_short_position'] = -1 + self.portfolio_trade_log.at[date, 'intraday_short_position']
                    else:
                        self.portfolio_trade_log.at[date, 'intraday_long_position'] = 1 + self.portfolio_trade_log.at[date, 'intraday_long_position']
                except:
                    pass
          
    def combine_strategy_pnls(self):
        
        pnl_columns = [i for i in self.portfolio_trade_log.columns if i.split('-')[0] == 'pnl']
        self.portfolio_trade_log['pnl'] = self.portfolio_trade_log[pnl_columns].sum(axis = 1)
    
    def combine_strategy_positions(self):
        
        position_columns = [i for i in self.portfolio_trade_log.columns if i.split('-')[0] == 'position']
        self.portfolio_trade_log['number_of_positions'] = self.portfolio_trade_log[position_columns].sum(axis = 1)
                
    def run_backtest(self, start_date='6-1-2016', end_date='6-1-2021'):
        
        self.start_date=start_date
        self.end_date=end_date
        self.get_daily_dates()
        self.add_strategy_positions_to_trade_log()
        self.get_portfolio_trade_log()
        self.combine_strategy_pnls()
        self.combine_strategy_positions()

        self.portfolio_trade_log['capital'] = 0
        for i in range(len(self.portfolio_trade_log)):
            if i == 0:
                if self.portfolio_trade_log['pnl'].iloc[i] == 0:
                    self.portfolio_trade_log['capital'].iloc[i] = self.starting_capital        
                else:
                    self.portfolio_trade_log['capital'].iloc[i] = ((self.quantity*self.lot_size) * self.portfolio_trade_log['pnl'].iloc[i]) + self.starting_capital
            else:
                if self.portfolio_trade_log['capital'].iloc[i-1] <= 0:
                    self.portfolio_trade_log['capital'].iloc[i-1] = 0
                    self.portfolio_trade_log['capital'].iloc[i] = 0
                else:
                    if self.portfolio_trade_log['pnl'].iloc[i] == 0:
                        self.portfolio_trade_log['capital'].iloc[i] = self.portfolio_trade_log['capital'].iloc[i-1]
                    else:
                        self.portfolio_trade_log['capital'].iloc[i] = ((self.quantity*self.lot_size) * self.portfolio_trade_log['pnl'].iloc[i]) + self.portfolio_trade_log['capital'].iloc[i-1]

        self.end_capital = self.portfolio_trade_log['capital'].iloc[-1]
    
    def get_weekly_monthly_quarterly_returns(self):
        
        self.portfolio_trade_log.reset_index(inplace=True)
        self.portfolio_trade_log['date'] = self.portfolio_trade_log['date'].apply(lambda x: str(x))
        self.portfolio_trade_log['date'] = self.portfolio_trade_log['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
        self.portfolio_trade_log.set_index('date', inplace=True)

        resampling_dict = {'capital': 'last'}

        self.df_weekly_pnl = self.portfolio_trade_log.resample('W').apply(resampling_dict)
        self.df_monthly_pnl = self.portfolio_trade_log.resample('M').apply(resampling_dict)
        self.df_quarterly_pnl = self.portfolio_trade_log.resample('Q').apply(resampling_dict)
        self.df_yearly_pnl = self.portfolio_trade_log.resample('Y').apply(resampling_dict)
    
    def get_return_metrics(self):
        
        self.no_of_years = ((self.portfolio_trade_log.index[-1] - self.portfolio_trade_log.index[0]).days/365)

        self.cagr = ((self.portfolio_trade_log['capital'].iloc[-1] / self.starting_capital)
                    ** (1/self.no_of_years) - 1)*100

        self.portfolio_trade_log['capital_drawdown'] = (
          (self.portfolio_trade_log['capital'] - self.portfolio_trade_log['capital'].cummax())/self.portfolio_trade_log['capital'].cummax()) * 100
        self.mdd_capital = self.portfolio_trade_log['capital_drawdown'].min()
        self.calmar_ratio = self.cagr/self.mdd_capital
        self.portfolio_trade_log.reset_index(inplace=True)
        df_equity_high = self.portfolio_trade_log[self.portfolio_trade_log['capital_drawdown'] == 0]
        df_equity_high['number_days_between_equity_highs'] = (
          df_equity_high['date'] - df_equity_high['date'].shift())

        try:
            if self.end_capital > self.starting_capital:
                self.recovery_days = df_equity_high['number_days_between_equity_highs'].apply(lambda x: x.days).max()
            else:
                self.recovery_days = np.nan
            
        except:
            self.recovery_days = np.nan
    
    def get_basic_metrics(self):
        
        self.no_of_trades = len(self.portfolio_trade_log[self.portfolio_trade_log['pnl'] != 0])

        self.no_of_wins = len(self.portfolio_trade_log[self.portfolio_trade_log['pnl'] > 0])
        self.win_prct = (self.no_of_wins / self.no_of_trades)*100
        self.average_profit_per_trade = self.portfolio_trade_log[self.portfolio_trade_log['pnl'] != 0]['pnl'].mean()
        self.median_of_trade = self.portfolio_trade_log[self.portfolio_trade_log['pnl'] != 0]['pnl'].median()
        self.max_profit = self.portfolio_trade_log['pnl'].max()
        self.min_profit = self.portfolio_trade_log['pnl'].min()
        self.gross_profit = self.portfolio_trade_log[self.portfolio_trade_log['pnl'] > 0]['pnl'].sum()
        self.gross_loss = self.portfolio_trade_log[self.portfolio_trade_log['pnl'] < 0]['pnl'].sum()
        self.net_profit = self.portfolio_trade_log['pnl'].sum()
        #self.buy_and_hold = self.df['close'].iloc[-1] - self.df['close'].iloc[0]
        self.profit_factor = self.gross_profit / self.gross_loss
        # oapf : outlier_adjusted_profit_factor
        self.oapf = (self.gross_profit - self.max_profit) / self.gross_loss
    
    def consecutive_pnl_metrics(self):
        
        df_cons = self.portfolio_trade_log[self.portfolio_trade_log['pnl']!=0]
        df_cons['continuous_wins'] = 0
        df_cons['continuous_losses'] = 0

        for i in range(1, len(df_cons)):
            if df_cons['pnl'].iloc[i-1] > 0:
                df_cons['continuous_wins'].iloc[i] = df_cons['continuous_wins'].iloc[i-1] + 1
            if df_cons['pnl'].iloc[i-1] < 0:
                df_cons['continuous_losses'].iloc[i] = df_cons['continuous_losses'].iloc[i-1] + 1
        self.portfolio_trade_log['continuous_wins'] = df_cons['continuous_wins']
        self.portfolio_trade_log['continuous_losses'] = df_cons['continuous_losses']
        
        self.consecutive_wins = self.portfolio_trade_log['continuous_wins'].max()
        self.consecutive_losses = self.portfolio_trade_log['continuous_losses'].max()
        
    def statistics_performance_metrics(self):
        
        
        self.t_test = stats.ttest_1samp(self.portfolio_trade_log[self.portfolio_trade_log['pnl'] != 0]['pnl'], 0)[0]
        self.p_value = (stats.ttest_1samp(self.portfolio_trade_log[self.portfolio_trade_log['pnl'] != 0]['pnl'], 0)[1])/2

        self.q1 = self.portfolio_trade_log[(self.portfolio_trade_log['pnl'] < self.median_of_trade) & (self.portfolio_trade_log['pnl'] != 0)]['pnl'].median()
        self.q3 = self.portfolio_trade_log[(self.portfolio_trade_log['pnl'] > self.median_of_trade) & (self.portfolio_trade_log['pnl'] != 0)]['pnl'].median()
        self.iqr = self.q3 - self.q1
        self.positive_outlier_level = self.q3 + (1.5*self.iqr)
        self.negative_outlier_level = self.q1 - (1.5*self.iqr)
        self.trades_pct_with_pnL_above_positive_outlier_level = 100 * (len(self.portfolio_trade_log[(self.portfolio_trade_log['pnl'] > self.positive_outlier_level) & (self.portfolio_trade_log['pnl'] != 0)]) / self.no_of_trades)
        self.trades_pct_with_pnL_below_negative_outlier_level = 100 * (len(self.portfolio_trade_log[(self.portfolio_trade_log['pnl'] < self.negative_outlier_level) & (self.portfolio_trade_log['pnl'] != 0)]) / self.no_of_trades)
        
    def get_backtest_metrics(self):
        
        self.get_return_metrics()
        self.get_basic_metrics()
        self.consecutive_pnl_metrics()
        self.statistics_performance_metrics()
        
        performance_dict = {'no_of_trades': [np.round(self.no_of_trades, 0)],
                            'no_of_wins': [np.round(self.no_of_wins, 0)],
                            'win_prct': [np.round(self.win_prct, 2)],
                            'average_profit_per_trade': [np.round(self.average_profit_per_trade, 2)],
                            'median_of_trade': [np.round(self.median_of_trade, 2)],
                            'net_profit': [np.round(self.net_profit, 2)],
                            'max_profit': [np.round(self.max_profit, 2)],
                            'min_profit': [np.round(self.min_profit, 2)],
                            'gross_profit': [np.round(self.gross_profit, 2)],
                            'gross_loss': [np.round(self.gross_loss, 2)],
                            'profit_factor': [abs(np.round(self.profit_factor, 2))],
                            'oapf': [abs(np.round(self.oapf, 2))],
                            'consecutive_wins': [np.round(self.consecutive_wins, 0)],
                            'consecutive_losses': [np.round(self.consecutive_losses, 0)],
                            't_test': [np.round(self.t_test, 2)],
                            'p_value': [np.round(self.p_value, 2)],
                            'q1': [np.round(self.q1, 2)],
                            'q3': [np.round(self.q3, 2)],
                            'iqr': [np.round(self.iqr, 2)],
                            'positive_outlier_level': [np.round(self.positive_outlier_level, 2)],
                            'negative_outlier_level': [np.round(self.negative_outlier_level, 2)],
                            'trades_pct_with_pnL_above_positive_outlier_level': [np.round(self.trades_pct_with_pnL_above_positive_outlier_level, 2)],
                            'trades_pct_with_pnL_below_negative_outlier_level': [np.round(self.trades_pct_with_pnL_below_negative_outlier_level, 2)],
                            'recovery_days': [np.round(self.recovery_days, 0)],
                            'starting_capital': [self.starting_capital],
                            'end_capital': [np.round(self.end_capital, 2)],
                            'no_of_years': [np.round(self.no_of_years, 0)],
                            'cagr': [np.round(self.cagr, 2)],
                            'mdd_capital': [abs(np.round(self.mdd_capital, 2))],
                            'calmar_ratio': [abs(np.round(self.calmar_ratio, 2))]}

        self.df_backtest_metrics = pd.DataFrame(performance_dict).T
        self.df_backtest_metrics.columns = ['performance_value']
    
    def get_equity_curve(self):
        
        self.portfolio_trade_log.set_index('date', inplace=True)
        self.portfolio_trade_log['capital'].plot()
        self.portfolio_trade_log.reset_index(inplace=True)